In [1]:
from base64 import b64encode, b64decode

def hex_to_base64(s):
    b64 = b64encode(bytes.fromhex(s)).decode()
    return b64

def base64_to_hex(b64):
    s2 = b64decode(b64.encode()).hex()
    return s2

In [2]:
def base64_encode(message_bytes):
    #message = "Python is fun"
    #message_bytes = message.encode('ascii')#converts into bytes
    base64_bytes = b64encode(message_bytes)#base64 encoding
    base64_message = base64_bytes.decode('utf-8')#bytes to ascii
    return base64_message

def base64_decode(string):
    base64_message = string
    base64_bytes = base64_message.encode('utf-8')
    message_bytes = b64decode(base64_bytes)#base64 decoding
    #message = message_bytes.decode('ascii')
    return message_bytes

In [3]:
def split_into_128bits(s):
    n = int(len(s)/32)
    ret = []
    j = 0
    for i in range(n):
        ret += [s[j:j+32]]
        j+=32
    return ret

In [4]:
from Crypto.Random import get_random_bytes
import sys
from aes10 import AES as AES_mod
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def encrypt_AES(plaintext, IV, key):
    """
    plaintext and IV as bytestring
    key in hex format
    """
    IV = base64_to_hex(base64_encode(IV))
    s = pad(plaintext, AES.block_size)
    s = base64_to_hex(base64_encode(s))
    res = split_into_128bits(s)
    new = AES_mod(key)
    #master_key = 0x3c7e155628aed2a6abf7158809cf4f3d
    plaintext = []
    encrypted = []
    n = len(res)
    for i in range(n):
        if(i==0):
            plaintext.append(int(res[i],16)^int(IV,16))
        else:
            #print(i,len(res))
            plaintext.append(int(res[i],16)^encrypted[i-1])
        encrypted.append(new.encrypt(plaintext[i]))
    return IV, encrypted

def decrypt_AES(IV, encrypted, key):
    n = len(encrypted)
    new = AES_mod(key)
    decrypted = []
    res = []
    for i in range(n):
        decrypted.append(new.decrypt(encrypted[i]))        
        if(i==0):
            res.append(hex(decrypted[i] ^ int(IV,16)))
        else:
            res.append(hex(decrypted[i]^encrypted[i-1]))
    s = "".join(i[2:] for i in res)
    s = bytes.fromhex(s)
    return(unpad(s,AES.block_size))


# plaintext1 = b'Python is the Best Language!'
# plaintext2 = b'Python is the Best Language. This has been the trend for the past 10 years or so!'
# IV = get_random_bytes(8)
# key = 0x3c7e155628aed2a6abf7158809cf4f3d
# enc_res = encrypt_AES(plaintext,IV,key)

# dec_res = decrypt_AES(enc_res[0],enc_res[1],key)
# print(dec_res)

def ciphertext_gen(plaintext, round_number):
    if(round_number == 1):
        from aes1 import AES as AES_mod
    elif(round_number == 2):
        from aes2 import AES as AES_mod
    elif(round_number == 3):
        from aes3 import AES as AES_mod
    elif(round_number == 4):
        from aes4 import AES as AES_mod
    elif(round_number == 5):
        from aes5 import AES as AES_mod
    elif(round_number == 6):
        from aes6 import AES as AES_mod
    elif(round_number == 7):
        from aes7 import AES as AES_mod
    elif(round_number == 8):
        from aes8 import AES as AES_mod
    elif(round_number == 9):
        from aes9 import AES as AES_mod
    elif(round_number == 10):
        from aes10 import AES as AES_mod
    enc_res = encrypt_AES(plaintext,IV,key)
    ciphertext = "".join(hex(i)[2:] for i in enc_res[1])
    return ciphertext
        #print("AES_CBC_round1_ciphertext : \n\nIV :\n",enc_res[0],"\n\nCIPHER TEXT : \n",ciphertext)
        #print("\nlen = ",len(ciphertext))

In [5]:
# fname = "/home/d/Downloads/link_status_search_with_ordering_real_csv/twom.csv"
# plaintext = open(fname,"r").read()[:-1]
# plaintext = plaintext[:2048].encode('ascii')

IV = get_random_bytes(8)
key = 0x3c7e155628aed2a6abf7158809cf4f3d

# plaintext, len(plaintext)

In [6]:
# def dict_update(g,plaintext):
#     plaintext = plaintext[:512].encode('ascii')
#     IV = get_random_bytes(8)
#     key = 0x3c7e155628aed2a6abf7158809cf4f3d
#     g['Round1'].append(ciphertext_gen(plaintext,1))
#     g['Round2'].append(ciphertext_gen(plaintext,2))
#     g['Round3'].append(ciphertext_gen(plaintext,3))
#     g['Round4'].append(ciphertext_gen(plaintext,4))
#     g['Round5'].append(ciphertext_gen(plaintext,5))
#     g['Round6'].append(ciphertext_gen(plaintext,6))
#     g['Round7'].append(ciphertext_gen(plaintext,7))
#     g['Round8'].append(ciphertext_gen(plaintext,8))
#     g['Round9'].append(ciphertext_gen(plaintext,9))
#     g['Round10'].append(ciphertext_gen(plaintext,10))
#     random_text = base64_to_hex(base64_encode(get_random_bytes(1024)))[:2000]
#     return g

In [19]:
def dict_update(g,plaintext):
    plaintext = plaintext.encode('utf-8')
    IV = get_random_bytes(8)
    key = 0x3c7e155628aed2a6abf7158809cf4f3d
    for i in range(1,11):
        g['text'].append(ciphertext_gen(plaintext,i)[:2000])
        g['class'].append(i)#class stands for round
    g['text'].append(base64_to_hex(base64_encode(get_random_bytes(1024)))[:2000])
    g['class'].append(0)#here 0th class represents random text
    return g

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("/home/d/datewise/apr/2/reviews2.csv")
len(df)

12500

In [14]:
df.iloc[0]['Text']

"The traditional Western is synonymous with wide open spaces, clearcut morality, inevitable storylines, the optimistic faith in a hero's ability to shape his own destiny, to escape his past. These qualities reflect directly the American sense of self, the self-shaping Dream, the pushing of boundaries and frontiers, which is why the genre is still alluded to by opportunistic politicians. With some noble exceptions (eg Wellman, Hawks), the Western was healthily free of neuroses or real anxiety. Anthony Mann changed all that forever, and this first foray into the genre is one of the most violent, vivid, complex, not to say exciting Westerns ever made.<br /><br />The traditional Western depends on a hero who exemplifies rugged wholesomeness, whatever misfortunes he may have had in the past, a supporter of order and right, who dominates the film, removes its obstacles, restores harmony in effect; and an obvious villain, who often, ironically, drives the plot, forces the hero into certain ac

In [20]:
g = {'text':[],'class':[]}

In [21]:
ldf = len(df)
for i in range(ldf):
    g = dict_update(g,df.iloc[i]['Text'])

In [22]:
df2 = pd.DataFrame(g)

In [15]:
df2.describe()

,class
count,66000.000000
mean,5.000000
std,3.162302
min,0.000000
25%,2.000000
50%,5.000000
75%,8.000000
max,10.000000


In [16]:
df2

,text,class
0,527c2ca4dff28e491b8797875058de94fd4026939d5f3f...,1
1,527c2ca4dff28e491b8797875058de94fd4026939d5f3f...,2
2,527c2ca4dff28e491b8797875058de94fd4026939d5f3f...,3
3,527c2ca4dff28e491b8797875058de94fd4026939d5f3f...,4
4,527c2ca4dff28e491b8797875058de94fd4026939d5f3f...,5
...,...,...
65995,2044f46e17ef4c9e2bc6fac984e55a23f40371fdafb553...,7
65996,2044f46e17ef4c9e2bc6fac984e55a23f40371fdafb553...,8
65997,2044f46e17ef4c9e2bc6fac984e55a23f40371fdafb553...,9
65998,2044f46e17ef4c9e2bc6fac984e55a23f40371fdafb553...,10


In [23]:
# df2.to_csv("allrounds2200.csv")
df2.to_csv("allrounds_reviews2.csv")

In [18]:
len(df2.iloc[100]['text'])

2025

In [56]:
#random_text
random_text = base64_to_hex(base64_encode(get_random_bytes(1024)))[:2025]
# new = base64_encode(get_random_bytes(10))
new, len(new)

('6a70e7798d23b3ade97fbc934924404d11af36cf633bd4d4e580452978268451c1c9fdd7cf52417cb11b04593150212c097b04bdd3505359d3b53141d73df36302ffc7bca674c532ca48a81ca739746ac60ad2e5508bcb02fa4d175925d870620c1a404eac24dc53ace16f9a8bba508cb639dfe204fe27f244aad4cd23a3851a22f9091451a95c568a40923c083a66c052bc9d161302593bd4b3706ea323a8f142910ccf787d3e36235d26aaf949fbbbe5ee44f8435a32557297c52067960027ccaf99825063a3c3106a0ce0e50aad460cc8fd4d9a7762b7b5ede394c178d355549224c06ffc936af8a22a7500a9f81cc4ed7800de8d9b229351024511b67925ed72b1b82db3dc34dd74e0079c06b65aa9e631e25428a008d86487f15d018e26fc8a10f7e0fc593d21373c7d7f1d32eca7a57648a3514189a675ba0325028f0af3c37f2a491f3b09f559bf49475506fbd33d5e8d95e505be957333c1826a8af9dfdc960d8bc2bd149cba743df0fda724ef72ea70a546d0d552755af6f58ab7ea6d5a1b62ccfd6f4af0b0ac4f2ae6a0bea9a038149ab90c2f29b9dfa38cdfa4e03216f5550f2c9a3dd5cb19df5c6950cadcc0b340f5f4efb5287aa4e964d04775e62d439ee7e2692057104f8b2f20a30ea06216107e1e92ed716a32ad5d4b3bd0b4df2eea88ca023e466720b51beb6f0ac4c599

In [39]:
len(df2['Round2'][0])

1053

In [55]:
plaintext = df.iloc[0]['text']
plaintext = plaintext[:1250].encode('ascii')
res = ciphertext_gen(plaintext,1)[:2025]
res, len(res)

('e13f68521594f8194af12694c7ee8f5e8770d9047d2cb76d44cfa7e4628bfda2caa6210e228ba33fa19aef5bb44827a87183df4f742ebd7d685a3f51d1ac7b323651df2931690783ca48f93d76a3506c7cac8e98e2619cbe846e72a5818fdfa851523c4bf550f76a1a550b425c98456ea21355dddb87242c11ed0c4e6c64707e438554a33e4e34961d7abf25e12b0ea17678b63e60f6d4bd542c5ebe50686196b637bb7d61326510642a8954f219d164dccae070ee7ca75f41f52ba4f64bceaa7214de8a9950e348011a34cf00da9b4cd9507a0fa5c66d55463760d1c8005ce241124b108d2d97c3f3e16494469f19d378519701f31e05bc29a5ff2bce660d45fe7de36d5f96445dccd80cc8ed4a3b4c1b32ff7398b98638d80f05f834cbd202fe908d0338ef2cebf83bf61c793f9843072617d8eb5bbe3a82dee9115a0ee7464dcac0d57c188a8d99e52a56a5b057e13a4ad20f491a286fcf8521277594561b6cf4376ace943044b54426915694058bec81d017c4a733a407d913c992f1352996037b0cfe43f11846b9825c4c4900cbc781d95d498259836c4987e753b0628836ba68a9b47daf89453b0c2df9a63a0ac6741288b420106f6135b69831b81c901f8899bc5a9a771867716a90f477e7ed657d9ef966f39c49427e5744d558b841e609187a4c649bb31c6d0b432ea0ffb54772c4

In [109]:
#round-9 ciphertext generation
from aes9 import AES as AES_mod
enc_res = encrypt_AES(plaintext,IV,key)
ciphertext = "".join(hex(i)[2:] for i in enc_res[1])

#print(enc_res)
print("AES_CBC_round9_ciphertext : \n\nIV :\n",enc_res[0],"\n\nCIPHER TEXT : \n",ciphertext)
print("\nlen = ",len(ciphertext))
# dec_res = decrypt_AES(enc_res[0],enc_res[1],key)
# print(dec_res)

AES_CBC_round9_ciphertext : 

IV :
 edba5d5e117998e6 

CIPHER TEXT : 
 178f491a0f82bf91408a805bcc6d93b68693e280eff777fe755632bcc417f4a721c5aee8badab291e524ed4579b8a1f9cb4043d83c96f1d0c94a4cd641038e14

len =  128


# Round - 10

In [110]:
from aes10 import AES as AES_mod
enc_res = encrypt_AES(plaintext,IV,key)
ciphertext = "".join(hex(i)[2:] for i in enc_res[1])
print("AES_CBC_round10_ciphertext : \n\nIV :\n",enc_res[0],"\n\nCIPHER TEXT : \n",ciphertext)
print("\nlen = ",len(ciphertext))

dec_res = decrypt_AES(enc_res[0],enc_res[1],key)
print("\nDECRYPTED TEXT:")
print(dec_res)

AES_CBC_round10_ciphertext : 

IV :
 edba5d5e117998e6 

CIPHER TEXT : 
 b4eb8266898525efda672cd6ea7431f0a85b736c4747d32051871b7922514c74198ad8469bda651ea349c045aaf3729fa903c05e6de1c6d0ab6c1a54dadf545a

len =  128

DECRYPTED TEXT:
b'hello,this is ABCDhello,this is ABCDhello,this is ABCD'
